In [19]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np

def load_json_file(filename):
    """Carga un archivo JSON y maneja posibles errores."""
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {filename}")
        return None
    except json.JSONDecodeError:
        print(f"Error: El archivo {filename} no es un JSON válido")
        return None
    except Exception as e:
        print(f"Error inesperado al cargar {filename}: {str(e)}")
        return None

def analyze_room_schedule(sala):
    """Analiza el horario de una sala específica."""
    # Crear matriz de horario vacía (5 días x 9 bloques)
    schedule = [[False for _ in range(9)] for _ in range(5)]
    
    # Marcar bloques ocupados
    days = {'Lunes': 0, 'Martes': 1, 'Miercoles': 2, 'Jueves': 3, 'Viernes': 4}
    for asignatura in sala['Asignaturas']:
        day_idx = days[asignatura['Dia']]
        block_idx = asignatura['Bloque'] - 1
        schedule[day_idx][block_idx] = True
    
    # Contar ventanas
    ventanas_total = 0
    bloques_ocupados_total = 0
    
    for day in schedule:
        ventanas_dia = 0
        ocupados = 0
        in_window = False
        
        for i in range(9):
            if day[i]:  # Bloque ocupado
                ocupados += 1
                if in_window:
                    ventanas_dia += 1
                    in_window = False
            else:  # Bloque vacío
                if ocupados > 0:
                    in_window = True
        
        if in_window and ocupados > 0:
            ventanas_dia += 1
            
        ventanas_total += ventanas_dia
        bloques_ocupados_total += ocupados
    
    return {
        'ventanas': ventanas_total,
        'bloques_ocupados': bloques_ocupados_total,
        'schedule': schedule
    }

def calculate_window_durations(horarios_salas):
    """Calcula la duración de las ventanas para cada sala."""
    try:
        window_stats = {}
        
        for sala in horarios_salas:
            codigo = sala['Codigo']
            ventanas = []
            
            # Crear matriz de ocupación
            schedule = [[False for _ in range(9)] for _ in range(5)]
            
            # Marcar bloques ocupados
            days = {'Lunes': 0, 'Martes': 1, 'Miercoles': 2, 'Jueves': 3, 'Viernes': 4}
            for asignatura in sala['Asignaturas']:
                day_idx = days[asignatura['Dia']]
                block_idx = asignatura['Bloque'] - 1
                schedule[day_idx][block_idx] = True
            
            # Encontrar ventanas y sus duraciones
            for day in schedule:
                window_size = 0
                prev_occupied = False
                for i in range(9):
                    if day[i]:  # Bloque ocupado
                        if window_size > 0:  # Si veníamos de una ventana
                            ventanas.append(window_size)
                            window_size = 0
                        prev_occupied = True
                    else:  # Bloque vacío
                        if prev_occupied:  # Solo contar ventanas entre bloques ocupados
                            window_size += 1
                if window_size > 0 and prev_occupied:  # Ventana al final del día
                    ventanas.append(window_size)
            
            # Agrupar ventanas por duración
            duration_counts = {}
            for duration in ventanas:
                if duration in duration_counts:
                    duration_counts[duration] += 1
                else:
                    duration_counts[duration] = 1
            
            window_stats[codigo] = {
                'duraciones': duration_counts,
                'total_bloques': sum(d * c for d, c in duration_counts.items())
            }
        
        return window_stats
        
    except Exception as e:
        print(f"Error al calcular las duraciones de ventanas: {str(e)}")
        return None

def amount_of_windows_lateral_chart(df_stats):
    """Crea un gráfico de barras horizontal para la cantidad de ventanas por sala."""
    try:
        plt.figure(figsize=(12, 8))
        
        # Ordenar el DataFrame por cantidad de ventanas de forma descendente
        df_sorted = df_stats.sort_values('ventanas', ascending=True)
        
        # Crear gráfico de barras horizontal
        ax = plt.gca()
        bars = ax.barh(df_sorted.index, df_sorted['ventanas'], color='#00629B')
        
        # Configurar el aspecto del gráfico
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        plt.title('Cantidad de Ventanas por Sala')
        plt.xlabel('Número de Ventanas')
        
        # Añadir etiquetas de valor al final de cada barra
        for i, bar in enumerate(bars):
            width = bar.get_width()
            ax.text(width, bar.get_y() + bar.get_height()/2, 
                   f' {int(width)}',
                   va='center')
        
        plt.tight_layout()
        plt.savefig('ventanas_cantidad.png')
        plt.close()
        
        return True
        
    except Exception as e:
        print(f"Error al crear el gráfico de cantidad de ventanas: {str(e)}")
        return False

def duration_of_windows_lateral_chart(window_stats):
    """Crea un gráfico de barras compuestas para las duraciones de ventanas."""
    try:
        # Colores IEEE (cíclicos)
        colors = ['#99C2E8', '#00629B', '#A9A9A9']
        
        # Preparar datos para el gráfico
        data = []
        max_duration = max(max(stats['duraciones'].keys()) 
                         for stats in window_stats.values() 
                         if stats['duraciones'])
        
        for sala, stats in window_stats.items():
            duraciones = stats['duraciones']
            total_bloques = stats['total_bloques']
            
            # Crear lista ordenada de duraciones
            dur_list = []
            for d in range(1, max_duration + 1):
                if d in duraciones:
                    dur_list.append((d, duraciones[d]))
            
            data.append({
                'sala': sala,
                'duraciones': dur_list,
                'total_bloques': total_bloques
            })
        
        # Ordenar por total de bloques
        data.sort(key=lambda x: x['total_bloques'])
        
        # Crear gráfico
        fig, ax = plt.subplots(figsize=(15, 8))
        
        # Variables para posicionamiento
        y_pos = np.arange(len(data))
        left = np.zeros(len(data))
        
        # Crear barras compuestas
        for duration in range(1, max_duration + 1):
            widths = []
            labels = []
            for i, item in enumerate(data):
                width = 0
                for dur, count in item['duraciones']:
                    if dur == duration:
                        width = count * duration
                        break
                widths.append(width)
                
                # Crear etiqueta si hay valor
                if width > 0:
                    labels.append(f'{width}h')
                else:
                    labels.append('')
            
            # Dibujar barra
            color = colors[(duration - 1) % len(colors)]
            bars = ax.barh(y_pos, widths, left=left, 
                         color=color, label=f'{duration}h')
            
            # Añadir etiquetas
            for idx, (width, label) in enumerate(zip(widths, labels)):
                if width > 0:
                    x = left[idx] + width/2
                    text_color = 'white' if color == '#00629B' else 'black'
                    ax.text(x, y_pos[idx], label,
                           ha='center', va='center', color=text_color)
            
            left += widths
        
        # Configurar aspecto
        ax.set_yticks(y_pos)
        ax.set_yticklabels([item['sala'] for item in data])
        
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        plt.title('Duración de Ventanas por Sala')
        plt.xlabel('Total de Bloques en Ventanas')
        
        # Añadir total al final de cada barra
        for idx, item in enumerate(data):
            if item['total_bloques'] > 0:
                ax.text(left[idx] + 0.5, y_pos[idx],
                       f'{item["total_bloques"]}h',
                       va='center')
        
        # Leyenda
        plt.legend(title='Duración de Ventana', 
                  bbox_to_anchor=(1.1, 1),  # Ajustar más a la izquierda
                  loc='upper left')
        
        plt.tight_layout()
        plt.savefig('ventanas_duracion.png', 
                   bbox_inches='tight')
        plt.close()
        
        return True
        
    except Exception as e:
        print(f"Error al crear el gráfico de duración de ventanas: {str(e)}")
        return False

def update_metrics_json(window_stats, df_stats):
    """Actualiza el archivo de métricas con toda la información."""
    try:
        metrics = {
            'estadisticas_por_sala': df_stats.to_dict(orient='index'),
            'estadisticas_duracion': window_stats
        }
        
        # Guardar archivo
        with open('metricas_compactacion.json', 'w', encoding='utf-8') as f:
            json.dump(metrics, f, ensure_ascii=False, indent=2)
        
        return True
        
    except Exception as e:
        print(f"Error al actualizar el archivo de métricas: {str(e)}")
        return False

def main():
    # 1. Cargar datos
    horarios_salas = load_json_file('../../agent_output/Horarios_salas.json')
    if horarios_salas is None:
        return
    
    # 2. Analizar cada sala
    results = {}
    
    for sala in horarios_salas:
        stats = analyze_room_schedule(sala)
        results[sala['Codigo']] = stats
    
    # 3. Crear DataFrame con resultados
    df_stats = pd.DataFrame.from_dict(results, orient='index')
    
    # 4. Calcular duraciones de ventanas
    window_stats = calculate_window_durations(horarios_salas)
    if window_stats is None:
        return
    
    # 5. Crear visualizaciones
    # 5.1 Gráfico de cantidad de ventanas
    if not amount_of_windows_lateral_chart(df_stats):
        return
        
    # 5.2 Gráfico de duración de ventanas
    if not duration_of_windows_lateral_chart(window_stats):
        return
    
    # 6. Actualizar archivo de métricas
    if update_metrics_json(window_stats, df_stats):
        print("\nAnálisis de compactación completado exitosamente")
    else:
        print("Error al completar el análisis de compactación")

if __name__ == "__main__":
    main()


Análisis de compactación completado exitosamente
